In [1]:
import glob
import pandas as pd
import numpy as np

在EDA中，我们主要观察特征的类型、总数量、非重复值数量、最大值、最小值、平均值、标准差、缺失值数量、缺失值占比、异常值数量和异常值占比。

在EDA中，我们对特征进行初筛，去除缺失值占比过高（>0.9）、非重复值数量过少（<= 1）以及字符型特征。

对于异常值，我们利用3sigma原理来判断。

In [3]:
def EDA_series(data):
    """
    input:
    data: series, 原始数据
    return:
    result: series, 原始数据EDA结果
    """
    result = {}
    result['count'] = len(data)
    result['missing_count'] = data.isnull().sum()
    result['missing_rate'] = result['missing_count']/ result['count']
    result['count_unqiue'] = len(data.value_counts(normalize=True))
    data.dropna(inplace=True)
    if data.dtype == 'object':
        result['type'] = 'categorical'
    else:
        result['type'] = 'numeric'
        result['max'] = data.max()
        result['min'] = data.min()
        result['mean'] = data.mean()
        result['std'] = data.std()
        zscore = (data - data.mean())/data.std()
        result['outlier_count'] = (zscore.abs() > 6).sum()
        result['outlier_rate'] = result['outlier_count'] / result['count']
    if result['count_unique'] <= 2:
        result['type'] = binary
    
    result = pd.Series(result)
    return result

In [4]:
def EDA_df(data):
    """
    input:
    data: dataframe, 原始数据
    return:
    result: dataframe, 原始数据EDA结果, index为特征名
    """
    result = []
    for column in data.columns.tolist():
        tmp = EDA_series(data[column])
        tmp.name = column
        result.append(tmp)
    result = pd.concat(result, axis=1).T
    columns_result = ['type', 'count', 'count_unique', 'max', 'min', 'mean', 'std', 'missing_count', 'missing_rate', 'outlier_count', 'outlier_rate']
    result = result[columns_result]
    return result